In [3]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.util import filter_spans
from spacy.symbols import nsubj, VERB
nlp = spacy.load('en_core_web_trf')

from tqdm import tqdm
import re
%matplotlib inline

In [4]:
strings = ['the Brown bears have 4 long claws and 2 short ears.',
           'The Brown bear has a brown fur.',
            'The Brown bear has a black nose.',
            'The Brown bear has a large nose.',
            'The Brown bear has a purple belly.',
            'Their feet are orange.',
            'The Brown bear has similar feet as the black bear.',
            'The claws have 5 sharp nails and are 8.5 cm long.',
            'The belly is bold and grey',
            'The nose is black and shiny',
            'It has dark ears',
            'legs long with curved nails'
          ]

In [5]:
displacy.render(nlp(strings[0]))

In [15]:
def extract_nouns(doc):
    
    triples = []
    
    for chunk in doc.noun_chunks:
        triples.append((chunk.root.lemma_),)
    
    return triples
    

In [18]:
triples = []
for string in strings:
    doc = nlp(string)

    triples.append(extract_nouns(doc))

In [19]:
triples

[['bear', 'claw', 'ear'],
 ['bear', 'fur'],
 ['bear', 'nose'],
 ['bear', 'nose'],
 ['bear', 'belly'],
 ['foot'],
 ['bear', 'foot', 'bear'],
 ['claw', 'nail'],
 ['belly'],
 ['nose'],
 ['it', 'ear'],
 ['leg', 'nail']]